# Libs and Inputs

In [18]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import time
from tqdm import tqdm
from GetIdxOutRadious import GetIdxOutRadious

LCOE_Max=range(300,30,-2) #LCOEs investigated
Radious=40 #Radious for the energy collection system

ShapeFileCoast="./GEO_data/ne_10m_coastline.shp"
ShapeFileStates="./GEO_data/ne_10m_admin_1_states_provinces_lines.shp"

WindKiteTrsData=np.load('PreprocessedData.npz',allow_pickle=True)

ResultsFileName='PortfolioOptimizationWindKite'

# Stage 1 Optimization

In [19]:
#LF kite and optimal kite design at each site location

## Prepare Data For the Optimization Model

In [20]:
#Wind Data
WindEnergy=WindKiteTrsData['WindEnergy']
WindLatLong=WindKiteTrsData['WindLatLong']
AnnualizedCostWind=WindKiteTrsData['AnnualizedCostWind']
MaxNumWindPerSite=WindKiteTrsData['MaxNumWindPerSite']

#Kite Data
KiteEnergy=WindKiteTrsData['KiteEnergy']
KiteLatLong=WindKiteTrsData['KiteLatLong']
AnnualizedCostKite=WindKiteTrsData['AnnualizedCostKite']
MaxNumKitesPerSite=WindKiteTrsData['MaxNumKitesPerSite']

#Transmission Data
AnnualizedCostTransmission=WindKiteTrsData['AnnualizedCostTransmission']
TransLatLong=WindKiteTrsData['TransLatLong']
EfficiencyTransmission=WindKiteTrsData['EfficiencyTransmission']
MaxPowerTransmission=float(WindKiteTrsData['MaxPowerTransmission'])

TimeStepHours=WindKiteTrsData['TimeStepHours'] #Number of hours for each time step

In [21]:
#Vectorize maximum number of turbines per site location per technology
Nu=np.concatenate((MaxNumWindPerSite,MaxNumKitesPerSite))

#Vectorize annualized cost for each site location and per technology
AnnCost=np.concatenate((AnnualizedCostWind,AnnualizedCostKite)) #Annualized cost [$/Year]

#Vectorize energy generation in each site location and energy resource
EnergyGeneration=np.concatenate((WindEnergy,KiteEnergy),axis=0) #MW ()

NumWindSites=WindEnergy.shape[0]
NumKiteSites=KiteEnergy.shape[0]

NumTrasmissionSites=TransLatLong.shape[0]

NumTimeSteps=WindEnergy.shape[1]

## Build Optimization Model Structure

In [22]:
MILP = ConcreteModel()

# Create Sets
MILP.SiteWind = RangeSet(0,NumWindSites-1)
MILP.SiteKite = RangeSet(0,NumKiteSites-1)
MILP.SiteTrs  = RangeSet(0,NumTrasmissionSites-1)

MILP.TimeSteps = RangeSet(0,NumTimeSteps-1)

# Create Variables
MILP.Y_Wind = Var(MILP.SiteWind, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location
MILP.Y_Kite = Var(MILP.SiteKite, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location

MILP.s = Var(MILP.SiteTrs, domain=Binary)# Binary variable to track the center of the energy collection system
MILP.Delta = Var(MILP.TimeSteps, domain=NonNegativeReals) #Curtailment variable

#Objective Function
def objective_rule(MILP):   
    EGWind=sum(MILP.Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP.SiteWind) #Energy generation from wind turbines
    EGKite=sum(MILP.Y_Kite[i]*np.average(KiteEnergy[i,:]) for i in MILP.SiteKite) #Energy generation from kite turbines

    TotalCurtailment=sum(MILP.Delta[t] for t in MILP.TimeSteps)/NumTimeSteps #Average curtailment MW

    Obj=(EGWind+EGKite-TotalCurtailment)#*24*365#Mwh/year
    return Obj

MILP.OBJ = Objective(rule = objective_rule, sense=maximize)

#Constraints

#Maximum number of turbines per site location wind
def MaxTurbinesCell_Wind_rule(MILP,i):
    return MILP.Y_Wind[i]<=MaxNumWindPerSite[i]
MILP.Turbines_Cell_Wind = Constraint(MILP.SiteWind, rule=MaxTurbinesCell_Wind_rule)

#Maximum number of turbines per site location kite
def MaxTurbinesCell_Kite_rule(MILP,i):
    return MILP.Y_Kite[i]<=MaxNumKitesPerSite[i]
MILP.Turbines_Cell_Kite = Constraint(MILP.SiteKite, rule=MaxTurbinesCell_Kite_rule)


#Curtailment constraint
def Curtailment_rule(MILP,t):
    EGWind=sum(MILP.Y_Wind[i]*WindEnergy[i,t] for i in MILP.SiteWind) #Energy generation from wind turbines
    EGKite=sum(MILP.Y_Kite[i]*KiteEnergy[i,t] for i in MILP.SiteKite) #Energy generation from kite turbines

    return -MILP.Delta[t]+ EGWind+ EGKite<=MaxPowerTransmission

MILP.Curtailment = Constraint(MILP.TimeSteps, rule=Curtailment_rule)


#---Choose center collection system - Start
MILP.ChooseOneCircle= Constraint(expr=sum(MILP.s[i] for i in MILP.SiteTrs)==1)

#Get the sites that are out of the radious of the center of the collection system
IdxOutWind=GetIdxOutRadious(TransLatLong, WindLatLong,Radious)
IdxOutKite=GetIdxOutRadious(TransLatLong, KiteLatLong,Radious)

def MaximumRadious(MILP,i):  
    SumWind_s=sum(MILP.Y_Wind[j] for j in IdxOutWind[i])
    SumKite_s=sum(MILP.Y_Kite[j] for j in IdxOutKite[i])

    return SumWind_s+SumKite_s<=(1-MILP.s[i])*300 # 300 is a big M for the maximum total number of turbines installed       

MILP.Maximum_Radious = Constraint(MILP.SiteTrs, rule=MaximumRadious)
#---Choose center collection system - End

#LCOE Target
def LCOETarget_S1(MILP,LCOE_Max):  
    EG_Wind=sum(MILP.Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP.SiteWind) #Average MW for wind
    EG_Kite=sum(MILP.Y_Kite[i]*np.average(KiteEnergy[i,:]) for i in MILP.SiteKite) #Average MW for kite
    TotalCurtailment=sum(MILP.Delta[i] for i in MILP.TimeSteps)/NumTimeSteps #Average curtailment MW
    
    MWh=(EG_Kite+EG_Wind-TotalCurtailment)*24*365 #MWh

    Cost_Wind=sum(MILP.Y_Wind[i]*AnnualizedCostWind[i] for i in MILP.SiteWind)
    Cost_Kite=sum(MILP.Y_Kite[i]*AnnualizedCostKite[i] for i in MILP.SiteKite)
    Cost_Transmission=sum(MILP.s[i]*AnnualizedCostTransmission[i] for i in MILP.SiteTrs)
    Cost=Cost_Wind+Cost_Kite+Cost_Transmission

    return Cost<=LCOE_Max*MWh # 300 is a big M for the total number of turbines installed       

#
opt = SolverFactory('gurobi', solver_io="python")
opt.options['mipgap'] = 0.05
#opt.options['max_iter'] = 500

## Solve Portfolio Optimization Stage 1 

In [23]:
SaveFeasibility, Save_LCOETarget, Save_LCOE_Achieved, SaveTotalMWAvg = list(), list(), list(), list()
SaveYWind, SaveYKite, SaveYTrans, SaveCurtail = list(), list(), list(), list()

LowestLCOE=10**10
for LCOE_Idx in tqdm(range(len(LCOE_Max))):
    LCOETarget=LCOE_Max[LCOE_Idx]
    
    if LCOETarget<LowestLCOE:    
        Bypass=0
        #Upperbound For the LCOE Activate Constraint
        LCOE_Target=LCOETarget_S1(MILP,LCOETarget)
        MILP.LCOE_Target = Constraint(rule=LCOE_Target)
        print("Running Model With LCOE= %.2f" % LCOETarget)
        
        try:
            results=opt.solve(MILP, tee=False)
        except:
            Bypass=1
            MILP.del_component(MILP.LCOE_Target)  
    
        if Bypass==0:
            if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
                SaveFeasibility.append(1)
                Save_LCOETarget.append(LCOETarget)
                
                Optimal_Y_Kite =np.array([MILP.Y_Kite.get_values()[j] for j in MILP.SiteKite])
                Optimal_Y_Wind =np.array([MILP.Y_Wind.get_values()[j] for j in MILP.SiteWind])
                Optimal_Y_Trans=np.array([MILP.s.get_values()[j] for j in MILP.SiteTrs])
                Curtailment=np.array([MILP.Delta.get_values()[j] for j in MILP.TimeSteps])

                SaveYWind.append(Optimal_Y_Wind)
                SaveYKite.append(Optimal_Y_Kite)
                SaveYTrans.append(Optimal_Y_Trans)
                SaveCurtail.append(Curtailment)

                #Current LCOE
                EG_Wind=np.sum(Optimal_Y_Wind*np.average(WindEnergy,axis=1)) #MW avg
                EG_Kite=np.sum(Optimal_Y_Kite*np.average(KiteEnergy,axis=1)) #MW avg
                TotalCurtailment=np.sum(Curtailment)/NumTimeSteps #MW avg
                MWh=(EG_Kite+EG_Wind-TotalCurtailment)*24*365 #MWh

                Cost_Wind  = sum(Optimal_Y_Wind*AnnualizedCostWind)
                Cost_Kite  = sum(Optimal_Y_Kite*AnnualizedCostKite)
                Cost_Trans = sum(Optimal_Y_Trans*AnnualizedCostTransmission)

                Cost=Cost_Wind+Cost_Kite+Cost_Trans
                CurrentLCOE=Cost/MWh
                
                Save_LCOE_Achieved.append(CurrentLCOE)
                SaveTotalMWAvg.append(value(MILP.OBJ))
                LowestLCOE=CurrentLCOE
                
                print("MW Wind: %.2f,\n MW Kite: %.2f,\n MW Curtailment: %.2f,\n MW Total: %.2f" % (EG_Wind,EG_Kite,TotalCurtailment,EG_Wind+EG_Kite-TotalCurtailment))

                
                #Delete constraint for its modification in the next step of the for loop
                MILP.del_component(MILP.LCOE_Target)
            
            else:# Something else is wrong
                MILP.del_component(MILP.LCOE_Target)
                SaveFeasibility.append(0)
                Save_LCOETarget.append(None)
                Save_LCOE_Achieved.append(None)
                SaveYWind.append(None)
                SaveYKite.append(None)
                SaveYTrans.append(None)
                SaveCurtail.append(None)    
                SaveTotalMWAvg.append(None)     

                break


  0%|          | 0/135 [00:00<?, ?it/s]

Running Model With LCOE= 300.00


  1%|          | 1/135 [00:05<11:44,  5.25s/it]

Running Model With LCOE= 298.00


  1%|▏         | 2/135 [00:10<11:37,  5.24s/it]

...

Running Model With LCOE= 206.00


 36%|███▌      | 48/135 [10:00<57:34, 39.71s/it]

Running Model With LCOE= 204.00


 36%|███▋      | 49/135 [10:03<41:02, 28.64s/it]

Running Model With LCOE= 202.00


In [ ]:
#Save Results
np.savez("./ResultsPortOpt/"+ResultsFileName +"Stage1LF"+ ".npz", 
        #Wind Data
        WindEnergy=WindEnergy,
        WindLatLong=WindLatLong,
        AnnualizedCostWind=AnnualizedCostWind,
        MaxNumWindPerSite=MaxNumWindPerSite,
        #Kite Data
        KiteEnergy=KiteEnergy,
        KiteLatLong=KiteLatLong,
        AnnualizedCostKite=AnnualizedCostKite,
        MaxNumKitesPerSite=MaxNumKitesPerSite,
        #Transmission Data
        AnnualizedCostTransmission=AnnualizedCostTransmission,
        TransLatLong=TransLatLong,
        EfficiencyTransmission=EfficiencyTransmission,
        MaxPowerTransmission=MaxPowerTransmission,
        TimeStepHours=TimeStepHours,
        #Solutions
        SaveFeasibility=SaveFeasibility,
        Save_LCOETarget=Save_LCOETarget,
        Save_LCOE_Achieved=Save_LCOE_Achieved,
        SaveYWind=SaveYWind,
        SaveYKite=SaveYKite,
        SaveYTrans=SaveYTrans,
        SaveCurtail=SaveCurtail,
        SaveTotalMWAvg=SaveTotalMWAvg)


# Stage 2 Optimization

In [ ]:
Save_LCOE_Achieved

[330.8561075260045]